# Detectors Comparison



O Detectors Comparison é um software coletor de estatísticas de algoritmos de detecção e extração de keypoints em fotografias turisticas.
Os dados estatísticos coletados são utilizados para avaliar o desempenho e precisão dos algoritmos: [ORB](), [BRISK](), [AKAZE](), [SIFT]() e [SURF]() em relação ao tempo de execução, quantidade de keypoints, quantidade de matches e porcentagem de acerto. A comparação é dividida em quatro categorias/casos/situações de pares de fotos: retratando o mesmo objeto na mesma escala, o mesmo objeto em escalas diferentes, objetos diferentes na mesma escala e objetos diferentes em escalas diferentes. Todos os pares de imagens se encontram relativamente no mesmo ângulo de visão.

Para realizarmos a tarefa proposta utilizamos como biblioteca principal a [OpenCV](), a qual fornece os algoritmos comparados. A demais bibliotecas ([NumPy](), [SciPy](), [SymPy](), [Time]() e [Matplotlib]()) funcionam como auxiliáres para a coleta de dados. Os dados, por sua vez, são salvos pelo [SQLite 3]().

In [1]:
import cv2
import sqlite3
import numpy as np
from scipy import stats
from sympy import Point, Line
from time import time, strftime
from matplotlib import pyplot as plt

## Processo de coleta de dados

A etapa principal do processo de coleta de cados consiste em:

+ Encontrar os Keypoints;
+ Encontrar os Matches através de uma busca completa;
+ Avaliar a taxa de acerto;
+ Calcular os ângulos entre as retas que passam pelo centro da imagem e cada keypoint com a reta horizontal (que passa pelo centro da imagem e o pixel mais à direita de mesma altura/coordenada y);
+ Calcular as diferenças entre os ângulos dos keypoints;
+ Calcular as razões entre as distâncias dos centros das imagens e seus keypoints, que chamamos de escala entre as imagens;
+ Calcular as médias e desvios padrão dos ângulos dos keypoints e das escalas;
+ Rotacionar a imagem da esquerda com a média dos ângulos dos keypoints;
+ Ampliar a imagem da esquerda com a escala;
+ Rencontrar os novos keypoints e matches;
+ Reavaliar a taxa de acerto;
+ Remover os falsos Matches:
    + Filtrar os ângulos e escalas menores que a média menos um desvio padrão ou maiores que a média mais um desvio padrão.
+ Recalcular as médias e os desvios padrão dos ângulos dos keypoints e das escalas;
+ Reaplicar a rotação e a ampliação com os novos valores de média;
+ Rencontrar os novos keypoints e matches;
+ Reavaliar a taxa de acerto;


In [46]:
NUM_OF_PAIRS = 1
TABLE_NAME = 'datas_{}'.format(strftime('%y%m%d_%H%M%S'))

In [29]:
# Finds the image's center
def image_center(image):
    return Point(image.shape[1] / 2, image.shape[0] / 2)

In [30]:
# Finds the angles between the horizontal axis
# and the lines passing through the image center
# and each keypoint
def g_find_kp_angles(image, kps):
    angles = []
    center = image_center(image)
    h_axis = Line(center, center.translate(center.x))
    for kp in kps:
        p = Point(kp.pt[0], kp.pt[1])
        kp_line = Line(center, p)
        angles.append(float(h_axis.angle_between(kp_line)))
    return angles

In [31]:
def angles_dif(angles_img1, angles_img2, matches):
    dif = []
    for match in matches :
        dif.append(angles_img1[match.queryIdx] - angles_img2[match.trainIdx])

    return dif

In [32]:
def remove_fake_matches(matches,dif_angles,angles_mean,angles_std,scales,scale_mean,scale_std):
    new_scales,new_dif_angles = [],[]
    for i in range(len(matches)):
        if dif_angles[i] < angles_mean + angles_std and dif_angles[i] > angles_mean - angles_std and scales[i] < scale_mean + scale_std and scales[i] > angles_mean - scale_std:
            new_scales.append(scales[i])
            new_dif_angles.append(dif_angles[i])
    return new_dif_angles,new_scales

In [33]:
# Finds the Key's points Angles
def find_kp_angles(kp1, kp2, matches, center1, center2):
    central_line = Line(center1, center2.translate(2 * center2.x))
    angles = []
    for match in matches:
        p1 = Point(kp1[match.queryIdx].pt[0], kp1[match.queryIdx].pt[1])
        p2 = Point(kp2[match.trainIdx].pt[0], kp2[match.trainIdx].pt[1])
        match_line = Line(p1, p2.translate(2 * center2.x))
        angles.append(float(central_line.angle_between(match_line)))
    return angles

In [34]:
def g_find_scale(image, kps):
    scale = []
    center = image_center(image)
    for kp in kps:
        p = Point(kp.pt[0], kp.pt[1])
        d = center.distance(p)
        scale.append(d)
    return scale


In [35]:
# Finds the ratio of the keypoints scale between images
def find_scale_ratios(img1, kp1, img2, kp2, matches):
    ratios = []
    scale1 = g_find_scale(img1, kp1)
    scale2 = g_find_scale(img2, kp2)
    for match in matches:
        # scale list preserves the ordering from keypoints list
        d1 = scale1[match.queryIdx]
        d2 = scale2[match.trainIdx]
        ratios.append(float(d1 / d2))
    return ratios

In [36]:
# Finds the Scale between images
def find_scale(kp1, kp2, matches, center1, center2):
    scale = []
    for match in matches:
        p1 = Point(kp1[match.queryIdx].pt[0], kp1[match.queryIdx].pt[1])
        p2 = Point(kp2[match.trainIdx].pt[0], kp2[match.trainIdx].pt[1])
        d1 = center1.distance(p1)
        d2 = center2.distance(p2)
        scale.append(float(d1 / d2))
    return scale

In [37]:
def affine_trans(img,angles,scale):
    center = image_center(img)
    m = cv2.getRotationMatrix2D((center.y, center.x), angles, scale)
    return cv2.warpAffine(img, m, (img.shape[1],img.shape[0]))

In [38]:
def save(conn,cursor,values):
    cursor.execute("""
      INSERT INTO {} (kp1,kp2,matches,time,anglesMean,anglesSD,scaleMean,scaleSD,technique,situation,pathImg1,pathImg2,phase)
      VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
      """.format(TABLE_NAME), values)
    conn.commit()

In [39]:
def ploting_image_pair(left,right):
    fig = plt.figure()
    fig.add_subplot(1,2,1)
    plt.imshow(left)
    fig.add_subplot(1,2,2)
    plt.imshow(right)
    plt.show()

In [40]:
def getStats(method,img1, img2):
    timeI = time()
    # find the keypoints and descriptors with ORB
    kp1, des1 = method.detectAndCompute(img1, None)
    kp2, des2 = method.detectAndCompute(img2, None)
    timeF = time()

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match descriptors. (query,train)
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key=lambda x: x.distance)

    return [kp1,kp2, matches, timeF - timeI]



In [41]:
def prep_values(img1,img2,method,name,case,pair):
    values = getStats(method,img1,img2)
    kp1,kp2,matches = values[0],values[1],values[2]
    values[0],values[1],values[2] = len(kp1),len(kp2),len(matches)

    angles_img1 = g_find_kp_angles(img1,kp1)
    angles_img2 = g_find_kp_angles(img2,kp2)
    angles_dif = angles_dif(angles_img1,angles_img2,matches)
    scales =  find_scale_ratios(img1, kp1, img2, kp2, matches)

    angles_mean = stats.tstd(angles_dif)
    angles_std = stats.tstd(angles_dif)

    scale_mean = stats.tmean(scales)
    scale_std = stats.tstd(scales)

    values.append(angles_mean)
    values.append(angles_std)
    values.append(scale_mean)
    values.append(scale_std)
    values.append(name)
    values.append(case)
    values.append('{}a.jpg'.format(pair))
    values.append('{}b.jpg'.format(pair))

    return angles_dif,scales,matches, values

In [47]:
def main():
    executeTimeI = time()
    conn = sqlite3.connect('banco.db')
    cursor = conn.cursor()
    cursor.execute(
      """CREATE TABLE {} (
            technique TEXT,
            situation TEXT,
            kp1 INTEGER,
            kp2 INTEGER,
            matches INTEGER,
            time FLOAT,
            anglesMean FLOAT,
            anglesSD FLOAT,
            scaleMean FLOAT,
            scaleSD FLOAT,
            pathImg1 TEXT,
            pathImg2 TEXT,
            phase INTEGER
      );""".format(TABLE_NAME)
    )

    # Initiate detectors
    # SIFT = cv2.xfeatures2d.SIFT_create()
    # SURF = cv2.xfeatures2d.SURF_create()
    ORB = cv2.ORB.create()
    # # KAZE = cv2.KAZE.create()
    # AKAZE = cv2.AKAZE.create()
    # BRISK = cv2.BRISK.create()

    methods = {
        # 'SIFT': SIFT,
        # 'SURF': SURF,
        'ORB': ORB,
        # 'KAZE': KAZE,
        # 'AKAZE': AKAZE,
        # 'BRISK': BRISK
    }

    cases = [
        'Same Object, Same Scale',
        # 'Same Object, Different Scale',
        # 'Different Object, Same Scale',
        # 'Different Object, Different Scale',
    ]

    for case in cases:
      print(case)
      for pair in range(NUM_OF_PAIRS):
        print('Pair {}/{}'.format(pair + 1, NUM_OF_PAIRS))
        img1 = cv2.imread('photos/{}/{}a.jpg'.format(case,pair),0)
        img2 = cv2.imread('photos/{}/{}b.jpg'.format(case,pair),0)
        for name, method in methods.items():
          print(name)
          print("Phase One: Compares unaltered images")
          angles_dif,scales,matches,original_values = prep_values(img1,img2,method,name,case,pair)
          original_values.append(1)
          save(conn, cursor,tuple(original_values))

          print('Phase two: Calculates the transformation')
          angles_mean = original_values[4]
          scale_mean = original_values[6]
          dst = gmt.affine_trans(img1,angles_mean,scale_mean)
          ploting_image_pair(dst,img2)
          _,_,_,values = prep_values(dst,img2,method,name,case,pair)
          values.append(2)

          save(conn, cursor,tuple(values))

          print("Phase three: Removes fake matches")
          angles_mean = original_values[4]
          angles_std = original_values[5]
          scale_mean = original_values[6]
          scale_std = original_values[7]

          angles_dif,scales = gmt.remove_fake_matches(matches,angles_dif,angles_mean,angles_std,scales,scale_mean,scale_std)

          angles_mean = stats.tstd(angles_dif)
          angles_std = stats.tstd(angles_dif)
          scale_mean = stats.tmean(scales)
          scale_std = stats.tstd(scales)

          dst = gmt.affine_trans(img1,angles_mean,scale_mean)
          ploting_image_pair(dst,img2)

          _,_,_,values = prep_values(dst,img2,method,name,case,pair)

          values.append(3)

          save(conn, cursor,tuple(values))

        del img1
        del img2
    conn.close()
    executeTimeF = time()
    print('Test executed in {} seconds'.format(executeTimeF-executeTimeI))


In [48]:
if(__name__ == '__main__'):
    main()

Same Object, Same Scale
Pair 1/1
ORB
Phase One: Compares unaltered images


UnboundLocalError: local variable 'angles_dif' referenced before assignment